In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
# List of all running backs on pro-football-reference
link = "https://www.pro-football-reference.com/players/rbindex.htm"

In [3]:
pfr_link = "https://www.pro-football-reference.com"

In [12]:
rb_df = pd.DataFrame()
count = 0

In [13]:
page = requests.get(link)
bs = BeautifulSoup(page.text, 'html.parser')
table = bs.find('tbody')

for row in table.find_all('tr')[count:]:
    count += 1
    
    nl = row.find('th') #contains name and link to player page
    name = nl.text
    
    player_link = pfr_link + nl.find('a').get('href')
    current = nl.find('strong') != None #boolean, true if player is still playing
    if current:
        continue
    
    pvdr = row.find_all('td') #position, value, drafted, retired
    pos = pvdr[0].text
    if pos[0:2] not in ["RB", "FB"]:
        continue
    
    print(count, name)
    
    AV = pvdr[1].text
    first_year = pvdr[2].text
    last_year = pvdr[3].text
    
    player_page = requests.get(player_link)
    player_bs = BeautifulSoup(player_page.text, 'html.parser')
    
    player_table = player_bs.find('div', id='switcher_rushing_and_receiving')
    if player_table:
        player_table = player_table.find('tbody')
    else:
        player_table = player_bs.find('div', id='switcher_receiving_and_rushing')
        if player_table:
            player_table = player_table.find('tbody')
        else:
            player_table = player_bs.find('div', id='div_rushing_and_receiving')
            if player_table:
                player_table = player_table.find('tbody')
            else:
                try:
                    player_table = player_bs.find('div', id='div_receiving_and_rushing').find('tbody')
                except AttributeError:
                    continue
                
    fantasy_points = []
    num_teams = 0
    for year in player_table.find_all('tr'):
        if num_teams > 0:
            num_teams -= 1
            continue
        
        stats = year.find_all("td")
        team = stats[1].text
        
        if "Missed season" in team:
            continue
        
        try:
            num_teams = int(team[0])
        except ValueError:
            num_teams = 0
        rush_yards = 0
        rush_td = 0
        rec = 0
        rec_yards = 0
        rec_td = 0
        
        try:
            rush_yards = float(year.find('td', attrs={"data-stat": "rush_yds"}).text)
        except ValueError:
            rush_yards = 0
        try:
            rush_td = float(year.find('td', attrs={"data-stat": "rush_td"}).text)
        except ValueError:
            rush_td = 0
        try:
            rec = float(year.find('td', attrs={"data-stat": "rec"}).text)
        except ValueError:
            rec = 0
        try:
            rec_yards = float(year.find('td', attrs={"data-stat": "rec_yds"}).text)
        except ValueError:
            rec_yards = 0
        try:
            rec_td = float(year.find('td', attrs={"data-stat": "rec_td"}).text)
        except ValueError:
            rec_td = 0
        
        points = rush_yards * 0.1     \
                    + rush_td * 6     \
                    + rec * 1         \
                    + rec_yards * 0.1 \
                    + rec_td * 6
        fantasy_points.append(round(points, 1))
    
    data_row = [name, pos, AV, first_year, last_year] + fantasy_points
    rb_df = pd.concat([rb_df, pd.DataFrame(data_row)], axis=1)
    
    if count % 150 == 0:
        time.sleep(3000)
    time.sleep(3)
    if count % 25 == 0:
        time.sleep(60)

1 Mark Ingram
2 Ezekiel Elliott
3 Alvin Kamara
4 Derrick Henry
5 Christian McCaffrey
6 Melvin Gordon
7 Dalvin Cook
8 Aaron Jones
9 Austin Ekeler
10 Latavius Murray
11 Nick Chubb
12 Giovani Bernard
13 Kareem Hunt
14 Joe Mixon
15 David Johnson
16 Leonard Fournette
17 Saquon Barkley
18 Josh Jacobs
19 Tevin Coleman
20 Kenyan Drake
21 James Conner
22 Jerick McKinnon
23 Jamaal Williams
24 Jonathan Taylor
25 Duke Johnson
26 Chris Carson
27 Miles Sanders
28 David Montgomery
29 Devin Singletary
30 Jordan Howard
31 Nyheim Hines
32 Tony Pollard
33 Matt Breida
34 Rex Burkhead
35 Sony Michel
36 Kyle Juszczyk
37 Mike Davis
38 Chase Edmonds
39 Marlon Mack
40 Raheem Mostert
41 D'Andre Swift
42 Gus Edwards
43 Antonio Gibson
44 Phillip Lindsay
45 Damien Williams
46 Ameer Abdullah
47 Brandon Bolden
48 Ronald Jones II
49 AJ Dillon
50 Ty Montgomery
51 Clyde Edwards-Helaire
52 Damien Harris
53 Najee Harris
54 Samaje Perine
55 James Robinson
56 Jeff Wilson
57 Alexander Mattison
58 Darrel Williams
59 D'Onta F

In [14]:
rb_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,Mark Ingram,RB,81,2011,2022,93.0,99.1,58.4,193.9,203.4,242.2,284.0,144.5,242.5,52.9,110.6,52.1
0,Ezekiel Elliott,RB,72,2016,2022,327.4,205.2,331.1,315.7,231.7,247.9,185.8,NaN,NaN,NaN,NaN,NaN
0,Alvin Kamara,RB,72,2017,2022,314.4,348.2,250.0,377.8,234.7,219.7,NaN,NaN,NaN,NaN,NaN,NaN
0,Derrick Henry,RB,64,2016,2022,105.7,135.0,202.8,300.6,335.1,187.1,304.6,NaN,NaN,NaN,NaN,NaN
0,Christian McCaffrey,RB,63,2017,2022,230.6,381.5,469.2,90.4,127.5,351.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Trey Ragas,RB,0,2021,2021,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Ronnie Rivers,RB,0,2022,2022,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Mekhi Sargent,RB,0,2021,2021,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Isaiah Spiller,RB,0,2022,2022,8.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
count

4738

In [16]:
rb_df = rb_df.T

In [17]:
rb_df = rb_df.reset_index(drop=True)
rb_df = rb_df.rename({0: "Name",
                      1: "Position",
                      2: "AV",
                      3: "First Year",
                      4: "Last Year",
                      5: "Y1",
                      6: "Y2",
                      7: "Y3",
                      8: "Y4",
                      9: "Y5",
                      10: "Y6",
                      11: "Y7",
                      12: "Y8",
                      13: "Y9",
                      14: "Y10",
                      15: "Y11",
                      16: "Y12",
                      17: "Y13",
                      18: "Y14",
                      19: "Y15",
                      20: "Y16",}, axis=1)

In [18]:
rb_df

,Name,Position,AV,First Year,Last Year,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10,Y11,Y12
0,Mark Ingram,RB,81,2011,2022,93.0,99.1,58.4,193.9,203.4,242.2,284.0,144.5,242.5,52.9,110.6,52.1
1,Ezekiel Elliott,RB,72,2016,2022,327.4,205.2,331.1,315.7,231.7,247.9,185.8,NaN,NaN,NaN,NaN,NaN
2,Alvin Kamara,RB,72,2017,2022,314.4,348.2,250.0,377.8,234.7,219.7,NaN,NaN,NaN,NaN,NaN,NaN
3,Derrick Henry,RB,64,2016,2022,105.7,135.0,202.8,300.6,335.1,187.1,304.6,NaN,NaN,NaN,NaN,NaN
4,Christian McCaffrey,RB,63,2017,2022,230.6,381.5,469.2,90.4,127.5,351.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,Trey Ragas,RB,0,2021,2021,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,Ronnie Rivers,RB,0,2022,2022,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,Mekhi Sargent,RB,0,2021,2021,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,Isaiah Spiller,RB,0,2022,2022,8.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
rb_df.to_csv("Current_RBData.csv")